---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # Open file
    f = open('university_towns.txt','r')
    university_towns_txt = f.readlines()
    f.close()
    
    # Set up dataframe
    columns = ["State","RegionName"]
    df = pd.DataFrame(columns=columns)    
    
    for line in university_towns_txt:
        line = line.rstrip()
        if "[edit]" in line:
            state_name = line[:-6]
        elif " (" in line:
            # Find where ( occurs
            ind_end = line.find(' (')
            region_name = line[:ind_end]
            df = df.append({'State': state_name,'RegionName': region_name},ignore_index=True)
        else:
            region_name = line
            df = df.append({'State': state_name,'RegionName': region_name},ignore_index=True)
            
    return df

In [4]:
df = get_list_of_university_towns()
# with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
#    print(df)
df.head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [5]:
uni_towns_list = [(row['State'], row['RegionName']) for index, row in df.iterrows()]

In [6]:
def past_2000(row):
    quarter = row['quarter']
    return int(quarter[:4]) > 1999

In [7]:
# Load GDP data as global variable
df_GDP_full = pd.read_excel('gdplev.xls',skiprows=8,header=None,usecols=[4,6],names=['quarter','GDP'])
df_GDP = df_GDP_full[df_GDP_full.apply(past_2000,axis=1)]

In [8]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    # Look for recession start time
    decline_flag = False
    for idx in df_GDP.index.values[1:]:    
        this_GDP = df_GDP.loc[idx]['GDP']
        previous_GDP = df_GDP.loc[idx-1]['GDP']

        if this_GDP < previous_GDP:
            if decline_flag is True:
                return df_GDP.loc[idx-1]['quarter']
                break
            else:
                decline_flag = True
        else:
            decline_flag = False
    
    return "No recession found"

In [9]:
get_recession_start()

'2008q3'

In [10]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    search_start = get_recession_start()
       
    # Look for recession end time
    increase_flag = False
    for idx in df_GDP[df_GDP['quarter'] > search_start].index.values[1:]:    
        this_GDP = df_GDP.loc[idx]['GDP']
        previous_GDP = df_GDP.loc[idx-1]['GDP']

        if this_GDP > previous_GDP:
            if increase_flag is True:
                return df_GDP.loc[idx]['quarter']
                break
            else:
                increase_flag = True
        else:
            increase_flag = False
        
    return "NO END FOUND"

In [11]:
get_recession_end()

'2009q4'

In [12]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    # Create a new dataframe that is df_GDP during the recession
    recession_start = get_recession_start()
    recession_end = get_recession_end()

    df_GDP_recession = df_GDP[df_GDP['quarter'] > recession_start]
    df_GDP_recession = df_GDP_recession[df_GDP_recession['quarter'] < recession_end]    
    
    # Locate the minimum, return that quarter
    return df_GDP_recession.loc[df_GDP_recession['GDP'].idxmin()]['quarter']

In [13]:
get_recession_bottom()

'2009q2'

In [14]:
def quarter_string(year, quarter):
    q_string = str(year) + 'q' + str(quarter)
    q_months = [str(year) + '-' + '{:02d}'.format(quarter*3-2),str(year) + '-' + '{:02d}'.format(quarter*3-1),str(year) + '-' + '{:02d}'.format(quarter*3)]
    
    return q_string, q_months

In [15]:
def build_df_zm():
    # Zillow-months
    df_zm = pd.read_csv('City_Zhvi_AllHomes.csv')
    cols_to_keep = list(df_zm.columns.values[1:3]) + list(df_zm.columns.values[51:])
    df_zm = df_zm[cols_to_keep]
    df_zm['2016-09'] = np.nan
    
    return df_zm

In [16]:
build_df_zm().head()

,RegionName,State,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,...,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09
0,New York,NY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,576200,578400,582200,588000,592200,592500,590200,588000,586400,NaN
1,Los Angeles,CA,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,...,560800,562800,565600,569700,574000,577800,580600,583000,585100,NaN
2,Chicago,IL,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,...,206900,206200,205800,206200,207300,208200,209100,211000,213000,NaN
3,Philadelphia,PA,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,...,121600,121800,123300,125200,126400,127000,127400,128300,129100,NaN
4,Phoenix,AZ,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,...,185300,186600,188000,189100,190200,191300,192800,194500,195900,NaN


In [17]:
def build_df_zq(df_zm):    
    # Make a list of all the quarters you are going to loop through
    q_strings = []
    for year in np.arange(2000, 2017, 1):
        for quarter in [1,2,3,4]:
            q_strings.append(quarter_string(year,quarter)[0])
    q_strings = q_strings[:-1] # Lose the last quarter
    
    # Set up a new dataframe with the final columns
    df_zq = pd.DataFrame(columns = ['RegionName','State'] + list(q_strings))
    df_zq['RegionName'] = df_zm['RegionName']
    df_zq['State'] = df_zm['State'].map(states)
    
    # Loop through all the months
    for q_string in q_strings:
        q_months = quarter_string(int(q_string[:4]),int(q_string[-1]))[1]
        df_zq[q_string] = df_zm[q_months].mean(axis=1)
    
    return df_zq

In [18]:
build_df_zq(build_df_zm()).head()

,RegionName,State,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
0,New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
1,Los Angeles,California,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
2,Chicago,Illinois,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
3,Philadelphia,Pennsylvania,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
4,Phoenix,Arizona,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [19]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df_zq = build_df_zq(build_df_zm())
    
    return df_zq.set_index(['State','RegionName'])

In [20]:
df_zq = convert_housing_data_to_quarters()
df_zq.head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [21]:
df_zq = convert_housing_data_to_quarters()
df_zq['price_ratio'] = df_zq[get_recession_start()]/df_zq[get_recession_bottom()]
df_zq = df_zq['price_ratio'].reset_index()
df_zq.head()

,State,RegionName,price_ratio
0,New York,New York,1.072844
1,California,Los Angeles,1.134332
2,Illinois,Chicago,1.055985
3,Pennsylvania,Philadelphia,1.006600
4,Arizona,Phoenix,1.151773


In [22]:
def is_uni(row):    
    return (row['State'],row['RegionName']) in uni_towns_list

In [41]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''

    # Load input
    df_zq = convert_housing_data_to_quarters()
    
    # Calculate ratio as price_ratio=quarter_before_recession/recession_bottom
    df_zq['price_ratio'] = df_zq[get_recession_start()]/df_zq[get_recession_bottom()]
    df_zq = df_zq['price_ratio'].reset_index()
    df_zq['uni'] = df_zq.apply(is_uni, axis=1) 
    
    is_unis = df_zq[df_zq['uni'] == True]['price_ratio'].dropna().values
    not_unis = df_zq[df_zq['uni'] == False]['price_ratio'].dropna().values
    
    ttest_results = ttest_ind(is_unis,not_unis)
    p = ttest_results[1]
    different = (p < .01)
    
    if np.average(is_unis) > np.average(not_unis):
        better = "non-university town"
    else:
        better = "university town"
    
    return (different, p, better)

run_ttest()

(False, 0.0054964273536938875, 'university town')